# Exploration of Several BioModels

## Preliminaries

In [1]:
import sys
sys.path.insert(0, "/home/ubuntu/symmathSBML")
import control as c
from controlSBML.control_sbml import ControlSBML
import controlSBML.constants as cn
try:
    from symmathSBML.symmath_sbml import SymmathSBML
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tellurium as te

import os
import urllib.request

# Helper Functions

In [2]:
def getSharedCodes(module_name="util"):
    """
    Obtains common codes from the github repository.

    Parameters
    ----------
    module_name: str
        name of the python module in the src directory
    """
    #url = "https://github.com/sys-bio/network-modeling-summer-school-2021/raw/main/src/%s.py" % moduleName
    url = "https://github.com/ModelEngineering/controlSBML/raw/main/controlSBML/%s.py" % module_name
    local_python = "python.py"
    _, _ = urllib.request.urlretrieve(url=url, filename=local_python)
    with open(local_python, "r") as fd:
        codeStr = "".join(fd.readlines())
    print(codeStr)
    exec(codeStr, globals())

# Acquire codes
getSharedCodes()

import numpy as np
import pandas as pd
import urllib.request


REPO_URL =  "https://github.com/ModelEngineering/controlSBML/raw/main"
MODULE_URLPAT =  REPO_URL + "/controlSBML/%s.py"  # Pattern for module URL
MODEL_URLPAT =  REPO_URL + "/models/%s"
MODEL_823_FILE = "biomodels_823.ant"
LOCAL_FILE = "local.txt"


def calculateMatrixDistance(mat1, mat2):
    """
    Calculates the distance between two matrices with the same shape.

    Parameters
    ----------
    mat1 - np.array
    mat2 - np.array
    
    Returns
    -------
    float
    """
    if np.shape(mat1) != np.shape(mat2):
        raise ValueError("Matrices must be the same shape.")
    return np.sum( (mat1 - mat2)**2)

def getSharedCodes(module_name="util"):
    """
    Obtains common codes from the github repository.

    Parameters
    ----------
    module_name: str
        name of the python module in the src directory
    """
    url = "https://github.com/ModelEngineering/controlSBML/raw/main/controlSBML/%s.py" % modul

In [3]:
def evaluateAccuracy(sbml_model, timepoints, suptitle="", is_plot=True, start_time=0, end_time=5, y_max=None, ymin=0):
    if isinstance(timepoints, float) or isinstance(timepoints, int):
        timepoints = [timepoints]
    ctlsb = ControlSBML(sbml_model)
    rr_df = ctlsb.simulateRoadrunner(start_time, end_time)
    nrow = len(timepoints)
    ncol = len(rr_df.columns)
    fig, axes = plt.subplots(nrow, ncol, figsize=(15, 5))
    axes = np.reshape(axes, (nrow, ncol))
    for irow, timepoint in enumerate(timepoints):
        linear_df = ctlsb.simulateLinearSystem(timepoint=timepoint, start_time=start_time, end_time=end_time)
        if y_max is None:
            ymax_val = max(linear_df.max().max(), rr_df.max().max())
        else:
            ymax_val = y_max
        for icol, column in enumerate(rr_df.columns):
            ax = axes[irow, icol]
            ax.plot(linear_df.index, linear_df[column], color="red")
            ax.plot(rr_df.index, rr_df[column], color="blue")
            ax.scatter([timepoint], [ymin], s=40, marker="o", color="g")
            ax.set_xlim([start_time, end_time])
            ax.set_ylim([0, ymax_val])
            if irow < nrow - 1:
                ax.set_xticklabels([])
            if irow == 0:
                ax.set_title(column, rotation=45)
                if icol == 0:
                    ax.text(start_time-3, 0.75*ymax_val, "Jacobian Time")
                    ax.legend(["linear", "nonlinear"])
            if icol > 0:
                ax.set_yticklabels([])
            else:
                ax.text(start_time-2, ymax_val/2, "%2.1f" % timepoint)
    plt.suptitle(suptitle)
    if is_plot:
        plt.show()
    else:
        plt.close()
    
# Tests
suptitle = "NONLINEAR1_MDL"
evaluateAccuracy(NONLINEAR1_MDL, [0, 1, 2, 3], suptitle=suptitle, start_time=2, y_max=20,  is_plot=True)
print("OK!")

NameError: name 'NONLINEAR1_MDL' is not defined

# Wolf Model

In [ ]:
rr = te.loadSBMLModel("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000206.2?filename=BIOMD0000000206_url.xml")

In [ ]:
rr.plot(rr.simulate())

In [ ]:
evaluateAccuracy(rr, [1, 2, 4, 5, 6], suptitle="Wolf", start_time=3, end_time=5, y_max=10)

# Model 193

In [ ]:
rr = te.loadSBMLModel("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000193.2?filename=BIOMD0000000193_url.xml")

In [ ]:
ctlsb_193 = ControlSBML("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000193.2?filename=BIOMD0000000193_url.xml")

In [ ]:
ctlsb_193.plotTrueModel(end_time=200, legend_crd=(1.6, 1))

In [ ]:
print(ctlsb_193.antimony)

In [ ]:
evaluateAccuracy(rr, [1, 2, 4, 5, 6], end_time=5, y_max=0.1)

# Model 816

In [ ]:
rr = te.loadSBMLModel("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000816.5?filename=model.xml")

In [ ]:
ctlsb_816 = ControlSBML(rr)

In [ ]:
print(ctlsb_816.antimony)

In [ ]:
ctlsb_816.plotTrueModel(y_max=5, legend_crd=(1.6, 1))

In [ ]:
#evaluateAccuracy(rr, [1, 2, 4, 5], end_time=5, ymax=5)
evaluateAccuracy(rr, [1], start_time=1.1, end_time=2, y_max=1)

# Model 823

Insulin levels impact outputs.

In [ ]:
ctlsb_823 = ControlSBML("https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000823.2?filename=Varusai2018.xml")

In [ ]:
print(ctlsb_823.antimony)

In [ ]:
MODEL = """
// Created by libAntimony v2.12.0
function Constant_flux__irreversible(v)
  v;
end

Constant_flux__irreversible is "Constant flux (irreversible)"

function Henri_Michaelis_Menten__irreversible(substrate, Km, V)
  V*substrate/(Km + substrate);
end

Henri_Michaelis_Menten__irreversible is "Henri-Michaelis-Menten (irreversible)"

function HMM_Mod(V, s, m, Km)
  V*s*m/(Km + s);
end

HMM_Mod is "HMM_Mod"

function Function_for_v11(k11ca, pmTORC1, DEPTOR, Km11a, pDEPTOR, k11cb, pmTORC2, Km11b)
  k11ca*pmTORC1*DEPTOR/(Km11a + pDEPTOR) + k11cb*pmTORC2*DEPTOR/(Km11b + DEPTOR);
end

Function_for_v11 is "Function_for_v11"

function Function_for_v5(k5ca, pIRS, Akt, Km5a, k5cb, pmTORC2, Km5b)
  k5ca*pIRS*Akt/(Km5a + Akt) + k5cb*pmTORC2*Akt/(Km5b + Akt);
end

Function_for_v5 is "Function_for_v5"


model *Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR()

  // Compartments and Species:
  compartment compartment_;
  species IR in compartment_, pIR in compartment_, IRS in compartment_, pIRS in compartment_;
  species iIRS in compartment_, Akt in compartment_, pAkt in compartment_;
  species mTORC1 in compartment_, pmTORC1 in compartment_, mTORC2 in compartment_;
  species pmTORC2 in compartment_, imTORC2 in compartment_, mTORC1_DEPTOR in compartment_;
  species mTORC2_DEPTOR in compartment_, DEPTOR in compartment_, pDEPTOR in compartment_;

  // Reactions:
  v1: IR => pIR; compartment_*Henri_Michaelis_Menten__irreversible(IR, Km1, V1);
  v2: pIR => IR; compartment_*Henri_Michaelis_Menten__irreversible(pIR, Km2, V2);
  v3: IRS => pIRS; compartment_*HMM_Mod(k3c, IRS, pIR, Km3);
  v4: pIRS => IRS; compartment_*Henri_Michaelis_Menten__irreversible(pIRS, Km4, V4);
  v5: Akt => pAkt; compartment_*Function_for_v5(k5ca, pIRS, Akt, Km5a, k5cb, pmTORC2, Km5b);
  v6: pAkt => Akt; compartment_*Henri_Michaelis_Menten__irreversible(pAkt, Km6, V6);
  v7: mTORC1 => pmTORC1; compartment_*HMM_Mod(k7c, mTORC1, pAkt, Km7);
  v8: pmTORC1 => mTORC1; compartment_*Henri_Michaelis_Menten__irreversible(pmTORC1, Km8, V8);
  v9: mTORC2 => pmTORC2; compartment_*HMM_Mod(k9c, mTORC2, pIR, Km9);
  v10: pmTORC2 => mTORC2; compartment_*Henri_Michaelis_Menten__irreversible(pmTORC2, Km10, V10);
  v11: DEPTOR => pDEPTOR; compartment_*Function_for_v11(k11ca, pmTORC1, DEPTOR, Km11a, pDEPTOR, k11cb, pmTORC2, Km11b);
  v12: pDEPTOR => DEPTOR; compartment_*Henri_Michaelis_Menten__irreversible(pDEPTOR, Km12, V12);
  v13: mTORC1 + DEPTOR -> mTORC1_DEPTOR; compartment_*(k13f*mTORC1*DEPTOR - k13r*mTORC1_DEPTOR);
  v14: mTORC2 + DEPTOR -> mTORC2_DEPTOR; compartment_*(k14f*mTORC2*DEPTOR - k14r*mTORC2_DEPTOR);
  v15: IRS => iIRS; compartment_*HMM_Mod(k15c, IRS, pmTORC1, Km15);
  v16: iIRS => IRS; compartment_*Henri_Michaelis_Menten__irreversible(iIRS, Km16, V16);
  v17:  => DEPTOR; compartment_*Constant_flux__irreversible(ks17);
  v18: pDEPTOR => ; compartment_*kd18*pDEPTOR;

  // Species initializations:
  IR = 50;
  pIR = 0;
  IRS = 100;
  pIRS = 0;
  iIRS = 0;
  Akt = 100;
  pAkt = 0;
  mTORC1 = 250;
  pmTORC1 = 0;
  mTORC2 = 200;
  pmTORC2 = 0;
  imTORC2 = 0;
  mTORC1_DEPTOR = 0;
  mTORC2_DEPTOR = 0;
  DEPTOR = 350;
  pDEPTOR = 0;

  // Compartment initializations:
  compartment_ = 1;

  // Variable initializations:
  V1 = 1;
  Km1 = 95;
  V2 = 1;
  Km2 = 35;
  k3c = 0.1;
  Km3 = 50;
  V4 = 1;
  Km4 = 500;
  k5ca = 0.05;
  Km5a = 7;
  k5cb = 1.5;
  Km5b = 4;
  V6 = 2;
  Km6 = 34;
  k7c = 0.1;
  Km7 = 2;
  V8 = 6;
  Km8 = 1;
  k9c = 0.3;
  Km9 = 160;
  V10 = 3;
  Km10 = 7;
  k11ca = 0.1;
  Km11a = 120;
  k11cb = 0.13;
  Km11b = 11;
  V12 = 4;
  Km12 = 7;
  k13f = 0.001;
  k13r = 0.006;
  k14f = 0.007;
  k14r = 0.006;
  k15c = 0.1;
  Km15 = 50;
  V16 = 1;
  Km16 = 50;
  ks17 = 0;
  kd18 = 0;

  // Other declarations:
  const compartment_, V1, Km1, V2, Km2, k3c, Km3, V4, Km4, k5ca, Km5a, k5cb;
  const Km5b, V6, Km6, k7c, Km7, V8, Km8, k9c, Km9, V10, Km10, k11ca, Km11a;
  const k11cb, Km11b, V12, Km12, k13f, k13r, k14f, k14r, k15c, Km15, V16;
  const Km16, ks17, kd18;

  // Unit definitions:
  unit volume = 1e-3 litre;
  unit substance = 1e-3 mole;

  // CV terms:
  compartment_ hypernym "http://identifiers.org/ncit/C48694"
  IR identity "http://identifiers.org/pr/PR:000009064"
  pIR hypernym "http://identifiers.org/pr/PR:000009064"
  IRS identity "http://identifiers.org/ncit/C28474"
  pIRS hypernym "http://identifiers.org/ncit/C28474"
  iIRS hypernym "http://identifiers.org/ncit/C28474"
  Akt identity "http://identifiers.org/pr/PR:000029189"
  pAkt hypernym "http://identifiers.org/pr/PR:000029189"
  mTORC1 hypernym "http://identifiers.org/ncit/C96314"
  pmTORC1 hypernym "http://identifiers.org/ncit/C96314"
  mTORC2 identity "http://identifiers.org/ncit/C96315"
  pmTORC2 hypernym "http://identifiers.org/ncit/C96315"
  imTORC2 hypernym "http://identifiers.org/ncit/C96315"
  mTORC1_DEPTOR part "http://identifiers.org/ncit/C96314"
  mTORC1_DEPTOR part "http://identifiers.org/ncit/C101595"
  mTORC2_DEPTOR part "http://identifiers.org/ncit/C101595"
  mTORC2_DEPTOR part "http://identifiers.org/ncit/mTORC2"
  DEPTOR identity "http://identifiers.org/ncit/C101595"
  pDEPTOR hypernym "http://identifiers.org/ncit/C101595"
  v1 hypernym "http://identifiers.org/go/GO:0016310"
  v2 hypernym "http://identifiers.org/go/GO:0016311"
  v3 hypernym "http://identifiers.org/go/GO:0016310"
  v4 hypernym "http://identifiers.org/go/GO:0016311"
  v5 hypernym "http://identifiers.org/go/GO:0016310"
  v6 hypernym "http://identifiers.org/go/GO:0016311"
  v7 hypernym "http://identifiers.org/go/GO:0016310"
  v8 hypernym "http://identifiers.org/go/GO:0016311"
  v9 hypernym "http://identifiers.org/go/GO:0016310"
  v10 hypernym "http://identifiers.org/go/GO:0016311"
  v11 hypernym "http://identifiers.org/go/GO:0016310"
  v12 hypernym "http://identifiers.org/go/GO:0016311"
  v13 hypernym "http://identifiers.org/ncit/C18469"
  v14 hypernym "http://identifiers.org/ncit/C18469"
  v15 hypernym "http://identifiers.org/ncit/C16983"
  v15 hypernym "http://identifiers.org/sbo/SBO:0000169"
  v16 hypernym "http://identifiers.org/ncit/C21018"
  v17 hypernym "http://identifiers.org/ncit/C80450"
  v18 hypernym "http://identifiers.org/ncit/C61559"
end

Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR is "Varusai2018 - Dynamic modelling of the mTOR signalling network reveals complex emergent behaviours conferred by DEPTOR"

Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR description "http://identifiers.org/pubmed/29330362"
Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR model_entity_is "http://identifiers.org/biomodels.db/MODEL1909250003",
                                                                                                                                       "http://identifiers.org/biomodels.db/BIOMD0000000823"
Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR property "http://identifiers.org/mamo/MAMO_0000046"
Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR property "http://identifiers.org/pw/PW:0000180"
Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR property "http://identifiers.org/ncit/C101595"
Varusai2018___Dynamic_modelling_of_the_mTOR_signalling_network_reveals_complex_emergent_behaviours_conferred_by_DEPTOR taxon "http://identifiers.org/taxonomy/9606"
"""

rr = te.loada(MODEL)
rr.plot(rr.simulate())

In [ ]:
MODEL_640 = """
// Created by libAntimony v2.12.0
function function_6(IRS_p, IRS_p_phos_by_p70_S6K_pT229_pT389, p70_S6K_pT229_pT389)
  IRS_p_phos_by_p70_S6K_pT229_pT389*IRS_p*p70_S6K_pT229_pT389;
end

function_6 is "Function for reaction_8"

function function_3(Amino_Acids, mTORC2, mTORC2_S2481_phos_by_Amino_Acids)
  mTORC2_S2481_phos_by_Amino_Acids*mTORC2*Amino_Acids;
end

function_3 is "Function for reaction_3"

function function_14(Akt_S473_phos_by_mTORC2_pS2481_second, Akt_pT308, mTORC2_pS2481)
  Akt_S473_phos_by_mTORC2_pS2481_second*Akt_pT308*mTORC2_pS2481;
end

function_14 is "Function for reaction_20"

function function_17(TSC1_TSC2, TSC1_TSC2_pS1387, mTORC1_pS2448, mTORC1_pS2448_dephos_by_TSC1_TSC2)
  mTORC1_pS2448_dephos_by_TSC1_TSC2*mTORC1_pS2448*(TSC1_TSC2 + TSC1_TSC2_pS1387);
end

function_17 is "Function for reaction_29"

function function_20(PI3K_p_PDK1, p70_S6K, p70_S6K_T229_phos_by_PI3K_p_PDK1_first)
  p70_S6K_T229_phos_by_PI3K_p_PDK1_first*p70_S6K*PI3K_p_PDK1;
end

function_20 is "Function for reaction_33"

function function_19(PI3K_variant_p, mTORC2, mTORC2_S2481_phos_by_PI3K_variant_p)
  mTORC2_S2481_phos_by_PI3K_variant_p*mTORC2*PI3K_variant_p;
end

function_19 is "Function for reaction_31"

function function_2(AMPK, AMPK_T172_phos_by_Amino_Acids, Amino_Acids)
  AMPK_T172_phos_by_Amino_Acids*AMPK*Amino_Acids;
end

function_2 is "Function for reaction_2"

function function_4(IR_beta, IR_beta_phos_by_Insulin, Insulin)
  IR_beta_phos_by_Insulin*IR_beta*Insulin;
end

function_4 is "Function for reaction_4"

function function_23(mTORC1_pS2448, p70_S6K_T389_phos_by_mTORC1_pS2448_second, p70_S6K_pT229)
  p70_S6K_T389_phos_by_mTORC1_pS2448_second*p70_S6K_pT229*mTORC1_pS2448;
end

function_23 is "Function for reaction_36"

function function_22(PI3K_p_PDK1, p70_S6K_T229_phos_by_PI3K_p_PDK1_second, p70_S6K_pT389)
  p70_S6K_T229_phos_by_PI3K_p_PDK1_second*p70_S6K_pT389*PI3K_p_PDK1;
end

function_22 is "Function for reaction_35"

function function_16(Akt_pT308, Akt_pT308_pS473, TSC1_TSC2, TSC1_TSC2_T1462_phos_by_Akt_pT308)
  TSC1_TSC2_T1462_phos_by_Akt_pT308*TSC1_TSC2*(Akt_pT308 + Akt_pT308_pS473);
end

function_16 is "Function for reaction_26"

function function_11(Akt, Akt_T308_phos_by_PI3K_p_PDK1_first, PI3K_p_PDK1)
  Akt_T308_phos_by_PI3K_p_PDK1_first*Akt*PI3K_p_PDK1;
end

function_11 is "Function for reaction_17"

function function_27(PRAS40_S183_phos_by_mTORC1_pS2448_second, PRAS40_pT246, mTORC1_pS2448)
  PRAS40_S183_phos_by_mTORC1_pS2448_second*PRAS40_pT246*mTORC1_pS2448;
end

function_27 is "Function for reaction_44"

function function_26(Akt_pT308, Akt_pT308_pS473, PRAS40_T246_phos_by_Akt_pT308_second, PRAS40_pS183)
  PRAS40_T246_phos_by_Akt_pT308_second*PRAS40_pS183*(Akt_pT308 + Akt_pT308_pS473);
end

function_26 is "Function for reaction_43"

function function_7(IRS, IRS_phos_by_p70_S6K_pT229_pT389, p70_S6K_pT229_pT389)
  IRS_phos_by_p70_S6K_pT229_pT389*IRS*p70_S6K_pT229_pT389;
end

function_7 is "Function for reaction_9"

function function_15(AMPK_pT172, TSC1_TSC2, TSC1_TSC2_S1387_phos_by_AMPK_pT172)
  TSC1_TSC2_S1387_phos_by_AMPK_pT172*TSC1_TSC2*AMPK_pT172;
end

function_15 is "Function for reaction_25"

function function_9(IR_beta_pY1146, PI3K_variant, PI3K_variant_phos_by_IR_beta_pY1146)
  PI3K_variant_phos_by_IR_beta_pY1146*PI3K_variant*IR_beta_pY1146;
end

function_9 is "Function for reaction_14"

function function_12(Akt, Akt_S473_phos_by_mTORC2_pS2481_first, mTORC2_pS2481)
  Akt_S473_phos_by_mTORC2_pS2481_first*Akt*mTORC2_pS2481;
end

function_12 is "Function for reaction_18"

function function_18(Amino_Acids, mTORC1, mTORC1_S2448_activation_by_Amino_Acids)
  mTORC1_S2448_activation_by_Amino_Acids*mTORC1*Amino_Acids;
end

function_18 is "Function for reaction_30"

function function_10(AMPK, AMPK_T172_phos, IRS_p)
  AMPK_T172_phos*AMPK*IRS_p;
end

function_10 is "Function for reaction_15"

function function_13(Akt_T308_phos_by_PI3K_p_PDK1_second, Akt_pS473, PI3K_p_PDK1)
  Akt_T308_phos_by_PI3K_p_PDK1_second*Akt_pS473*PI3K_p_PDK1;
end

function_13 is "Function for reaction_19"

function function_8(IRS_p, PI3K_PDK1, PI3K_PDK1_phos_by_IRS_p)
  PI3K_PDK1_phos_by_IRS_p*PI3K_PDK1*IRS_p;
end

function_8 is "Function for reaction_12"

function function_21(mTORC1_pS2448, p70_S6K, p70_S6K_T389_phos_by_mTORC1_pS2448_first)
  p70_S6K_T389_phos_by_mTORC1_pS2448_first*p70_S6K*mTORC1_pS2448;
end

function_21 is "Function for reaction_34"

function function_5(IRS, IRS_phos_by_IR_beta_pY1146, IR_beta_pY1146)
  IRS_phos_by_IR_beta_pY1146*IRS*IR_beta_pY1146;
end

function_5 is "Function for reaction_7"

function function_24(PRAS40, PRAS40_S183_phos_by_mTORC1_pS2448_first, mTORC1_pS2448)
  PRAS40_S183_phos_by_mTORC1_pS2448_first*PRAS40*mTORC1_pS2448;
end

function_24 is "Function for reaction_41"

function function_25(Akt_pT308, Akt_pT308_pS473, PRAS40, PRAS40_T246_phos_by_Akt_pT308_first)
  PRAS40_T246_phos_by_Akt_pT308_first*PRAS40*(Akt_pT308 + Akt_pT308_pS473);
end

function_25 is "Function for reaction_42"

function function_1(Amino_Acids, IRS, IRS_phos_by_Amino_Acids)
  IRS_phos_by_Amino_Acids*IRS*Amino_Acids;
end

function_1 is "Function for reaction_1"


model *MODEL1702270000()

  // Compartments and Species:
  compartment Cell;
  species IR_beta in Cell, IR_beta_pY1146 in Cell, IR_beta_refractory in Cell;
  species IRS in Cell, IRS_p in Cell, IRS_pS636 in Cell, AMPK in Cell, AMPK_pT172 in Cell;
  species Akt in Cell, Akt_pT308 in Cell, Akt_pS473 in Cell, Akt_pT308_pS473 in Cell;
  species TSC1_TSC2 in Cell, TSC1_TSC2_pT1462 in Cell, TSC1_TSC2_pS1387 in Cell;
  species mTORC1 in Cell, mTORC1_pS2448 in Cell, mTORC2 in Cell, mTORC2_pS2481 in Cell;
  species p70_S6K in Cell, p70_S6K_pT229 in Cell, p70_S6K_pT389 in Cell, p70_S6K_pT229_pT389 in Cell;
  species PRAS40 in Cell, PRAS40_pT246 in Cell, PRAS40_pS183 in Cell, PRAS40_pT246_pS183 in Cell;
  species PI3K_variant in Cell, PI3K_variant_p in Cell, PI3K_PDK1 in Cell;
  species PI3K_p_PDK1 in Cell, $Insulin in Cell, $Amino_Acids in Cell, $IR_beta_pY1146_obs in Cell;
  species $IRS_pS636_obs in Cell, $AMPK_pT172_obs in Cell, $Akt_pT308_obs in Cell;
  species $Akt_pS473_obs in Cell, $TSC1_TSC2_pS1387_obs in Cell, $mTOR_pS2448_obs in Cell;
  species $mTOR_pS2481_obs in Cell, $p70_S6K_pT229_obs in Cell, $p70_S6K_pT389_obs in Cell;
  species $PRAS40_pT246_obs in Cell, $PRAS40_pS183_obs in Cell;

  // Assignment Rules:
  Insulin := piecewise(0, time < 0, 0); # Has impact
  Amino_Acids := piecewise(0, time < 0, 5);
  IR_beta_pY1146_obs := IR_beta_pY1146;
  IRS_pS636_obs := IRS_pS636;
  AMPK_pT172_obs := AMPK_pT172;
  Akt_pT308_obs := Akt_pT308 + Akt_pT308_pS473;
  Akt_pS473_obs := Akt_pS473 + Akt_pT308_pS473;
  TSC1_TSC2_pS1387_obs := TSC1_TSC2_pS1387;
  mTOR_pS2448_obs := mTORC1_pS2448;
  mTOR_pS2481_obs := mTORC2_pS2481;
  p70_S6K_pT229_obs := p70_S6K_pT229 + p70_S6K_pT229_pT389;
  p70_S6K_pT389_obs := p70_S6K_pT389 + p70_S6K_pT229_pT389;
  PRAS40_pT246_obs := PRAS40_pT246 + PRAS40_pT246_pS183;
  PRAS40_pS183_obs := PRAS40_pS183 + PRAS40_pT246_pS183;

  // Reactions:
  reaction_1: IRS => IRS_p; Cell*function_1(Amino_Acids, IRS, IRS_phos_by_Amino_Acids);
  reaction_2: AMPK => AMPK_pT172; Cell*function_2(AMPK, AMPK_T172_phos_by_Amino_Acids, Amino_Acids);
  reaction_3: mTORC2 => mTORC2_pS2481; Cell*function_3(Amino_Acids, mTORC2, mTORC2_S2481_phos_by_Amino_Acids);
  reaction_4: IR_beta => IR_beta_pY1146; Cell*function_4(IR_beta, IR_beta_phos_by_Insulin, Insulin);
  reaction_5: IR_beta_pY1146 => IR_beta_refractory; Cell*IR_beta_pY1146_dephos*IR_beta_pY1146;
  reaction_6: IR_beta_refractory => IR_beta; Cell*IR_beta_ready*IR_beta_refractory;
  reaction_7: IRS => IRS_p; Cell*function_5(IRS, IRS_phos_by_IR_beta_pY1146, IR_beta_pY1146);
  reaction_8: IRS_p => IRS_pS636; Cell*function_6(IRS_p, IRS_p_phos_by_p70_S6K_pT229_pT389, p70_S6K_pT229_pT389);
  reaction_9: IRS => IRS_pS636; Cell*function_7(IRS, IRS_phos_by_p70_S6K_pT229_pT389, p70_S6K_pT229_pT389);
  reaction_10: IRS_pS636 => IRS; Cell*IRS_pS636_turnover*IRS_pS636;
  reaction_11: PI3K_p_PDK1 => PI3K_PDK1; Cell*PI3K_p_PDK1_dephos*PI3K_p_PDK1;
  reaction_12: PI3K_PDK1 => PI3K_p_PDK1; Cell*function_8(IRS_p, PI3K_PDK1, PI3K_PDK1_phos_by_IRS_p);
  reaction_13: PI3K_variant_p => PI3K_variant; Cell*PI3K_variant_p_dephos*PI3K_variant_p;
  reaction_14: PI3K_variant => PI3K_variant_p; Cell*function_9(IR_beta_pY1146, PI3K_variant, PI3K_variant_phos_by_IR_beta_pY1146);
  reaction_15: AMPK => AMPK_pT172; Cell*function_10(AMPK, AMPK_T172_phos, IRS_p);
  reaction_16: AMPK_pT172 => AMPK; Cell*AMPK_pT172_dephos*AMPK_pT172;
  reaction_17: Akt => Akt_pT308; Cell*function_11(Akt, Akt_T308_phos_by_PI3K_p_PDK1_first, PI3K_p_PDK1);
  reaction_18: Akt => Akt_pS473; Cell*function_12(Akt, Akt_S473_phos_by_mTORC2_pS2481_first, mTORC2_pS2481);
  reaction_19: Akt_pS473 => Akt_pT308_pS473; Cell*function_13(Akt_T308_phos_by_PI3K_p_PDK1_second, Akt_pS473, PI3K_p_PDK1);
  reaction_20: Akt_pT308 => Akt_pT308_pS473; Cell*function_14(Akt_S473_phos_by_mTORC2_pS2481_second, Akt_pT308, mTORC2_pS2481);
  reaction_21: Akt_pT308 => Akt; Cell*Akt_pT308_dephos_first*Akt_pT308;
  reaction_22: Akt_pS473 => Akt; Cell*Akt_pS473_dephos_first*Akt_pS473;
  reaction_23: Akt_pT308_pS473 => Akt_pS473; Cell*Akt_pT308_dephos_second*Akt_pT308_pS473;
  reaction_24: Akt_pT308_pS473 => Akt_pT308; Cell*Akt_pS473_dephos_second*Akt_pT308_pS473;
  reaction_25: TSC1_TSC2 => TSC1_TSC2_pS1387; Cell*function_15(AMPK_pT172, TSC1_TSC2, TSC1_TSC2_S1387_phos_by_AMPK_pT172);
  reaction_26: TSC1_TSC2 => TSC1_TSC2_pT1462; Cell*function_16(Akt_pT308, Akt_pT308_pS473, TSC1_TSC2, TSC1_TSC2_T1462_phos_by_Akt_pT308);
  reaction_27: TSC1_TSC2_pS1387 => TSC1_TSC2; Cell*TSC1_TSC2_pS1387_dephos*TSC1_TSC2_pS1387;
  reaction_28: TSC1_TSC2_pT1462 => TSC1_TSC2; Cell*TSC1_TSC2_pT1462_dephos*TSC1_TSC2_pT1462;
  reaction_29: mTORC1_pS2448 => mTORC1; Cell*function_17(TSC1_TSC2, TSC1_TSC2_pS1387, mTORC1_pS2448, mTORC1_pS2448_dephos_by_TSC1_TSC2);
  reaction_30: mTORC1 => mTORC1_pS2448; Cell*function_18(Amino_Acids, mTORC1, mTORC1_S2448_activation_by_Amino_Acids);
  reaction_31: mTORC2 => mTORC2_pS2481; Cell*function_19(PI3K_variant_p, mTORC2, mTORC2_S2481_phos_by_PI3K_variant_p);
  reaction_32: mTORC2_pS2481 => mTORC2; Cell*mTORC2_pS2481_dephos*mTORC2_pS2481;
  reaction_33: p70_S6K => p70_S6K_pT229; Cell*function_20(PI3K_p_PDK1, p70_S6K, p70_S6K_T229_phos_by_PI3K_p_PDK1_first);
  reaction_34: p70_S6K => p70_S6K_pT389; Cell*function_21(mTORC1_pS2448, p70_S6K, p70_S6K_T389_phos_by_mTORC1_pS2448_first);
  reaction_35: p70_S6K_pT389 => p70_S6K_pT229_pT389; Cell*function_22(PI3K_p_PDK1, p70_S6K_T229_phos_by_PI3K_p_PDK1_second, p70_S6K_pT389);
  reaction_36: p70_S6K_pT229 => p70_S6K_pT229_pT389; Cell*function_23(mTORC1_pS2448, p70_S6K_T389_phos_by_mTORC1_pS2448_second, p70_S6K_pT229);
  reaction_37: p70_S6K_pT229 => p70_S6K; Cell*p70_S6K_pT229_dephos_first*p70_S6K_pT229;
  reaction_38: p70_S6K_pT389 => p70_S6K; Cell*p70_S6K_pT389_dephos_first*p70_S6K_pT389;
  reaction_39: p70_S6K_pT229_pT389 => p70_S6K_pT389; Cell*p70_S6K_pT229_dephos_second*p70_S6K_pT229_pT389;
  reaction_40: p70_S6K_pT229_pT389 => p70_S6K_pT229; Cell*p70_S6K_pT389_dephos_second*p70_S6K_pT229_pT389;
  reaction_41: PRAS40 => PRAS40_pS183; Cell*function_24(PRAS40, PRAS40_S183_phos_by_mTORC1_pS2448_first, mTORC1_pS2448);
  reaction_42: PRAS40 => PRAS40_pT246; Cell*function_25(Akt_pT308, Akt_pT308_pS473, PRAS40, PRAS40_T246_phos_by_Akt_pT308_first);
  reaction_43: PRAS40_pS183 => PRAS40_pT246_pS183; Cell*function_26(Akt_pT308, Akt_pT308_pS473, PRAS40_T246_phos_by_Akt_pT308_second, PRAS40_pS183);
  reaction_44: PRAS40_pT246 => PRAS40_pT246_pS183; Cell*function_27(PRAS40_S183_phos_by_mTORC1_pS2448_second, PRAS40_pT246, mTORC1_pS2448);
  reaction_45: PRAS40_pS183 => PRAS40; Cell*PRAS40_pS183_dephos_first*PRAS40_pS183;
  reaction_46: PRAS40_pT246 => PRAS40; Cell*PRAS40_pT246_dephos_first*PRAS40_pT246;
  reaction_47: PRAS40_pT246_pS183 => PRAS40_pT246; Cell*PRAS40_pS183_dephos_second*PRAS40_pT246_pS183;
  reaction_48: PRAS40_pT246_pS183 => PRAS40_pS183; Cell*PRAS40_pT246_dephos_second*PRAS40_pT246_pS183;

  // Species initializations:
  IR_beta = 50;
  IR_beta_pY1146 = 0;
  IR_beta_refractory = 0;
  IRS = 150;  # Doubling has an impact
  IRS_p = 0;
  IRS_pS636 = 0;
  AMPK = 50;
  AMPK_pT172 = 0;
  Akt = 300;
  Akt_pT308 = 0;
  Akt_pS473 = 0;
  Akt_pT308_pS473 = 0;
  TSC1_TSC2 = 50;
  TSC1_TSC2_pT1462 = 0;
  TSC1_TSC2_pS1387 = 0;
  mTORC1 = 100;
  mTORC1_pS2448 = 0;
  mTORC2 = 100;
  mTORC2_pS2481 = 0;
  p70_S6K = 300;
  p70_S6K_pT229 = 0;
  p70_S6K_pT389 = 0;
  p70_S6K_pT229_pT389 = 0;
  PRAS40 = 20;
  PRAS40_pT246 = 0;
  PRAS40_pS183 = 0;
  PRAS40_pT246_pS183 = 0;
  PI3K_variant = 50;
  PI3K_variant_p = 0;
  PI3K_PDK1 = 150; # Doubling and tripling has an impact
  PI3K_p_PDK1 = 0;

  // Compartment initializations:
  Cell = 1;

  // Variable initializations:
  IRS_phos_by_Amino_Acids = 0.0331672;
  AMPK_T172_phos_by_Amino_Acids = 17.6284;
  mTORC2_S2481_phos_by_Amino_Acids = 0.0268658;
  IR_beta_phos_by_Insulin = 0.0203796;
  IR_beta_pY1146_dephos = 0.493514;
  IR_beta_ready = 323.611;
  IRS_phos_by_IR_beta_pY1146 = 2.11894;
  IRS_p_phos_by_p70_S6K_pT229_pT389 = 0.338859859949792;
  IRS_phos_by_p70_S6K_pT229_pT389 = 0.0863775267376444;
  IRS_pS636_turnover = 25;
  AMPK_T172_phos = 0.490602;
  AMPK_pT172_dephos = 165.704;
  Akt_S473_phos_by_mTORC2_pS2481_first = 1.31992e-05;
  Akt_S473_phos_by_mTORC2_pS2481_second = 0.159093;
  Akt_T308_phos_by_PI3K_p_PDK1_first = 7.47437;
  Akt_T308_phos_by_PI3K_p_PDK1_second = 7.47345;
  Akt_pT308_dephos_first = 88.9654;
  Akt_pT308_dephos_second = 88.9639;
  Akt_pS473_dephos_first = 0.376999;
  Akt_pS473_dephos_second = 0.380005;
  TSC1_TSC2_S1387_phos_by_AMPK_pT172 = 0.00175772;
  TSC1_TSC2_T1462_phos_by_Akt_pT308 = 1.52417;
  TSC1_TSC2_pS1387_dephos = 0.25319;
  TSC1_TSC2_pT1462_dephos = 147.239;
  mTORC1_pS2448_dephos_by_TSC1_TSC2 = 0.00869774;
  mTORC1_S2448_activation_by_Amino_Acids = 0.0156992;
  mTORC2_pS2481_dephos = 1.42511;
  mTORC2_S2481_phos_by_PI3K_variant_p = 0.120736;
  p70_S6K_T229_phos_by_PI3K_p_PDK1_first = 0.0133520172873009;
  p70_S6K_T229_phos_by_PI3K_p_PDK1_second = 1.00000002814509e-06;
  p70_S6K_T389_phos_by_mTORC1_pS2448_first = 0.00261303413778722;
  p70_S6K_T389_phos_by_mTORC1_pS2448_second = 0.110720890919343;
  p70_S6K_pT229_dephos_first = 1.00000012897033e-06;
  p70_S6K_pT229_dephos_second = 0.159201353240651;
  p70_S6K_pT389_dephos_first = 1.10036057608758;
  p70_S6K_pT389_dephos_second = 1.10215267954479;
  PRAS40_S183_phos_by_mTORC1_pS2448_first = 0.15881;
  PRAS40_S183_phos_by_mTORC1_pS2448_second = 0.0683009;
  PRAS40_T246_phos_by_Akt_pT308_first = 0.279344;
  PRAS40_T246_phos_by_Akt_pT308_second = 0.279401;
  PRAS40_pS183_dephos_first = 1.8706;
  PRAS40_pS183_dephos_second = 1.88453;
  PRAS40_pT246_dephos_first = 11.8759;
  PRAS40_pT246_dephos_second = 11.876;
  PI3K_p_PDK1_dephos = 0.18913343080532;
  PI3K_PDK1_phos_by_IRS_p = 0.000187226757782201;
  PI3K_variant_p_dephos = 0.108074886441184;
  PI3K_variant_phos_by_IR_beta_pY1146 = 0.000549027801822575;
  scale_IR_beta_pY1146_obs = 1;
  scale_IRS_pS636_obs = 1;
  scale_AMPK_pT172_obs = 1;
  scale_Akt_pT308_obs = 1;
  scale_Akt_pS473_obs = 1;
  scale_TSC1_TSC2_pS1387_obs = 1;
  scale_mTOR_pS2448_obs = 1;
  scale_mTOR_pS2481_obs = 1;
  scale_p70_S6K_pT229_obs = 1;
  scale_p70_S6K_pT389_obs = 1;
  scale_PRAS40_pT246_obs = 1;
  scale_PRAS40_pS183_obs = 1;

  // Other declarations:
  const Cell, IRS_phos_by_Amino_Acids, AMPK_T172_phos_by_Amino_Acids, mTORC2_S2481_phos_by_Amino_Acids;
  const IR_beta_phos_by_Insulin, IR_beta_pY1146_dephos, IR_beta_ready, IRS_phos_by_IR_beta_pY1146;
  const IRS_p_phos_by_p70_S6K_pT229_pT389, IRS_phos_by_p70_S6K_pT229_pT389;
  const IRS_pS636_turnover, AMPK_T172_phos, AMPK_pT172_dephos, Akt_S473_phos_by_mTORC2_pS2481_first;
  const Akt_S473_phos_by_mTORC2_pS2481_second, Akt_T308_phos_by_PI3K_p_PDK1_first;
  const Akt_T308_phos_by_PI3K_p_PDK1_second, Akt_pT308_dephos_first, Akt_pT308_dephos_second;
  const Akt_pS473_dephos_first, Akt_pS473_dephos_second, TSC1_TSC2_S1387_phos_by_AMPK_pT172;
  const TSC1_TSC2_T1462_phos_by_Akt_pT308, TSC1_TSC2_pS1387_dephos, TSC1_TSC2_pT1462_dephos;
  const mTORC1_pS2448_dephos_by_TSC1_TSC2, mTORC1_S2448_activation_by_Amino_Acids;
  const mTORC2_pS2481_dephos, mTORC2_S2481_phos_by_PI3K_variant_p, p70_S6K_T229_phos_by_PI3K_p_PDK1_first;
  const p70_S6K_T229_phos_by_PI3K_p_PDK1_second, p70_S6K_T389_phos_by_mTORC1_pS2448_first;
  const p70_S6K_T389_phos_by_mTORC1_pS2448_second, p70_S6K_pT229_dephos_first;
  const p70_S6K_pT229_dephos_second, p70_S6K_pT389_dephos_first, p70_S6K_pT389_dephos_second;
  const PRAS40_S183_phos_by_mTORC1_pS2448_first, PRAS40_S183_phos_by_mTORC1_pS2448_second;
  const PRAS40_T246_phos_by_Akt_pT308_first, PRAS40_T246_phos_by_Akt_pT308_second;
  const PRAS40_pS183_dephos_first, PRAS40_pS183_dephos_second, PRAS40_pT246_dephos_first;
  const PRAS40_pT246_dephos_second, PI3K_p_PDK1_dephos, PI3K_PDK1_phos_by_IRS_p;
  const PI3K_variant_p_dephos, PI3K_variant_phos_by_IR_beta_pY1146, scale_IR_beta_pY1146_obs;
  const scale_IRS_pS636_obs, scale_AMPK_pT172_obs, scale_Akt_pT308_obs, scale_Akt_pS473_obs;
  const scale_TSC1_TSC2_pS1387_obs, scale_mTOR_pS2448_obs, scale_mTOR_pS2481_obs;
  const scale_p70_S6K_pT229_obs, scale_p70_S6K_pT389_obs, scale_PRAS40_pT246_obs;
  const scale_PRAS40_pS183_obs;

  // Unit definitions:
  unit volume = dimensionless;
  unit time_unit = 60 second;
  unit substance = dimensionless;

  // Display Names:
  time_unit is "time";

  // SBO terms:
  Cell.sboTerm = 290

  // CV terms:
  Cell property "http://identifiers.org/fma/FMA:68646"
  IR_beta hypernym "http://identifiers.org/uniprot/P06213"
  IRS hypernym "http://identifiers.org/uniprot/P35569"
  AMPK hypernym "http://identifiers.org/uniprot/Q5EG47"
end

MODEL1702270000 is "DallePezze2016 - Activation of AMPK and mTOR by amino acids"

MODEL1702270000 model_entity_is "http://identifiers.org/biomodels.db/MODEL1702270000"
MODEL1702270000 model_entity_is "http://identifiers.org/biomodels.db/BIOMD0000000640"
MODEL1702270000 description "http://identifiers.org/pubmed/27869123"
MODEL1702270000 taxon "http://identifiers.org/taxonomy/40674"
MODEL1702270000 property "http://identifiers.org/go/GO:0010506",
                         "http://identifiers.org/mamo/MAMO_0000046"
"""
rr = te.loada(MODEL_640)
rr.plot(rr.simulate(0, 20))

In [ ]:
symmath_823 = SymmathSBML(ctlsb_823.antimony)

In [ ]:
symmath_823.getJacobianSymbols()

In [ ]:
symmath_823.system

In [ ]:
ctlsb_823.set({"ks17": 1})
ctlsb_823.plotTrueModel(legend_crd=(1.4, 1))

In [ ]:
ctlsb_823.set({"ks17": 0, "IR": 50, "Km2": 500 })
ctlsb_823.plotTrueModel(legend_crd=(1.4, 1))

In [ ]:
ctlsb_823.get("Km1")

In [ ]:
evaluateAccuracy(ctlsb_823.roadrunner, [1, 2, 4, 5], end_time=5)

# Model 640

In [ ]:
url_640 = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000640.2?filename=BIOMD0000000640_url.xml"

In [ ]:
symmath = SymmathSBML(url_640)

In [ ]:
symmath.getSystem()

In [ ]:
pd.set_option("max_colwidth", 300)
symmath.getJacobian()

In [ ]:
def runModel(url, legend_crd=(1.6, 1), is_plot=True, is_accuracy=True, **kwargs):
    ctlsb = ControlSBML(url)
    ctlsb.plotTrueModel(legend_crd=legend_crd, is_plot=is_plot, **kwargs)
    if is_accuracy:
        evaluateAccuracy(ctlsb.roadrunner, [1, 2, 4, 5], end_time=5, is_plot=is_plot)
    return ctlsb
    
# Tests
ctlsb_640a = runModel(url_640, is_plot=False, is_accuracy=False, end_time=20)

In [ ]:
ctlsb_640b = runModel(MODEL_640, is_plot=False, end_time=20, is_accuracy=False)

In [ ]:
ctlsb_640a.setTime(0)
jaca= ctlsb_640a.jacobian

ctlsb_640a.setTime(5)
jacb= ctlsb_640a.jacobian

np.sqrt(np.sum(jaca - jacb)**2)

In [ ]:
ctlsb_640a.plotTrueModel(end_time=20, legend_crd=(1.6, 1),y_max=30)

In [ ]:
ctlsb_640a.plotTrueModel(end_time=20, legend_crd=(1.6, 1),y_max=30)

In [ ]:
evaluateAccuracy(ctlsb_640a.roadrunner, [1, 2, 4, 5], end_time=20, y_max=30)

In [ ]:
#evaluateAccuracy(ctlsb_640.roadrunner, [1, 2, 4, 5], end_time=20, y_max=30)
evaluateAccuracy(MODEL_640, [1, 2, 4, 5], end_time=20, y_max=30)

In [ ]:
print(ctlsb_640.antimony)

In [ ]:
rr = te.loada(MODEL_640)
rr.plot(rr.simulate(0, 20))

## Notebook on this model
1. Background and content of the model
1. Accuracy of linear approximation
1. Effectors